In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math

from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression,Ridge,Lasso
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold,train_test_split 
from sklearn.metrics         import r2_score,mean_squared_error
from sklearn.ensemble        import RandomForestRegressor,  GradientBoostingRegressor 
from xgboost                 import XGBRegressor 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\train_processedWithLog1pTarget.csv")

In [4]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\test_processed.csv")

In [5]:
train_df.isnull().sum()

latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
log_(price+1)                        0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [6]:
train_df = train_df.drop(['latitude']

SyntaxError: unexpected EOF while parsing (<ipython-input-6-d865eeaa9c2c>, line 1)

In [7]:
train_df_with_dummies = train_df

In [8]:
X=train_df_with_dummies.drop(['log_(price+1)'],axis=1)
y=train_df_with_dummies['log_(price+1)']

In [9]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [10]:
#random forest
rf_model = RandomForestRegressor(random_state=33)
param_grid = dict(n_estimators=range(10,100,10),max_depth=range(3,20),min_samples_split=range(50,500,20),min_samples_leaf=range(25,75,10),max_leaf_nodes=range(8,32,2))
grid = RandomizedSearchCV(rf_model,param_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=5,random_state=33)
grid.fit(X_train,y_train);


print('Train RMSLE:',np.sqrt(mean_squared_error(y_train, grid.best_estimator_.predict(X_train))))
print('Test RMSLE:',np.sqrt(mean_squared_error(y_test, grid.best_estimator_.predict(X_test))));

Train RMSLE: 0.4666649239860458
Test RMSLE: 0.4816031265383796


In [11]:
y_pred_rf = grid.best_estimator_.predict(X_test)
#rmse_train_rf = math.sqrt(mean_squared_error(y_train,y_pred_rf))
rmse_rf = math.sqrt(mean_squared_error(y_test,y_pred_rf))

In [12]:
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, grid.best_estimator_.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, grid.best_estimator_.predict(X_test))))

Train RMSE: 0.4666649239860458
Test RMSE: 0.4816031265383796


In [13]:
rmse_rf

0.4816031265383796

In [14]:
np.sqrt(mean_squared_error(y_train,np.exp(grid.best_estimator_.predict(X_train))-1)), np.sqrt(mean_squared_error(y_test,np.exp(grid.best_estimator_.predict(X_test))-1))

(139.04790544932055, 138.89749837206728)

In [50]:
#feature selection
from sklearn.feature_selection import RFE

In [51]:
def feature_selection(predictors,target,number_of_features,model):

    models = model()
    rfe = RFE(models,number_of_features)
    rfe = rfe.fit(X,y)
    feature_ranking = pd.Series(rfe.ranking_, index=X.columns)
    plt.show()
    print('Features  to be selected for {} are:'.format(str(i[0])))
    print(feature_ranking[feature_ranking.values==1].index.tolist())
    print('===='*30)

In [52]:
# Choosing the models. If you want to specify additional models, kindly specify them as a key-value pair as shown below.
models = {'Random Forest':RandomForestRegressor}
# Selecting 8 number of features
for i in models.items():
    feature_selection(X,y,15,i[1])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Features  to be selected for Random Forest are:
['latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365', 'neighbourhood_group_Bronx', 'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan', 'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island', 'room_type_Entire home/apt', 'room_type_Private room', 'room_type_Shared room']


## TEST DATA

In [59]:
test_df

,id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,1525602,40.66751,-73.95867,2,121,1.70,1,39,0,1,0,0,0,1,0,0
1,30430185,40.75655,-73.99690,1,18,2.49,30,364,0,0,1,0,0,0,1,0
2,21354525,40.69252,-73.99121,1,87,4.29,1,108,0,1,0,0,0,0,1,0
3,35995074,40.77292,-73.90101,9,0,0.01,2,365,0,0,0,1,0,0,1,0
4,34392081,40.71863,-73.94980,1,28,14.00,2,20,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14664,27730191,40.59446,-73.94289,2,1,0.59,1,36,0,1,0,0,0,1,0,0
14665,29676417,40.74319,-73.98074,2,1,0.16,1,365,0,0,1,0,0,0,0,1
14666,1813829,40.82519,-73.94530,1,93,1.34,3,358,0,0,1,0,0,0,1,0
14667,25017403,40.72238,-73.99466,1,15,1.12,8,34,0,0,1,0,0,1,0,0


In [60]:
test_id_col = test_df['id']

In [61]:
test_df.isnull().sum()

id                                   0
latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [65]:
test_df_with_dummies = test_df

In [66]:
test_df_with_dummies = test_df_with_dummies.drop(['id'],axis=1)

In [67]:
test_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [68]:
test_df_with_dummies['price'] = grid.predict(test_df_with_dummies)

In [69]:
test_df_with_dummies['price'] = np.exp(test_df_with_dummies['price'])-1

In [70]:
submissions_RF = pd.concat([test_id_col, test_df_with_dummies['price']], axis=1) 


In [71]:
submissions_RF.to_csv('submission_RF_with_cat.csv', index=False) 

In [74]:
submissions_RF

,id,price
0,1525602,126.707272
1,30430185,186.326177
2,21354525,81.509515
3,35995074,59.140908
4,34392081,78.895334
...,...,...
14664,27730191,125.410865
14665,29676417,225.049054
14666,1813829,65.578894
14667,25017403,193.119879


In [75]:
submissions_RF.isnull().sum()

id       0
price    0
dtype: int64